# Segmenting and Clustering Neighborhoods in Toronto - Second Part

In this notebook we are going to get the latitude and longitude for each Neighborhood in Toronto using Geocoder Python

### Data load

First we need to load the data we gathered in the previous step.

In [1]:
import csv
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
pip install geocoder

     |████████████████████████████████| 102kB 18.5MB/s ta 0:00:01
     |████████████████████████████████| 92kB 21.4MB/s eta 0:00:01
     |████████████████████████████████| 829kB 8.0MB/s eta 0:00:01:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [4]:
import csv
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load data

In [10]:
neighborhoods_data=pd.read_csv('data_Toronto.csv') 

In [11]:
neighborhoods_data.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood
0,0,M1B,Scarborough,"Malvern , Rouge"
1,1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,3,M1G,Scarborough,Woburn
4,4,M1H,Scarborough,Cedarbrae


## Tranform the data into a pandas dataframe

In [12]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [13]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [22]:
neighborhoods = neighborhoods_data.drop(columns=['Unnamed: 0'])
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Then let's loop through the data and fill the dataframe one row at a time.

In [26]:
neighborhoods["Latitude"] = ""
neighborhoods["Longitude"] = ""

In [27]:
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",,
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",,
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


Let's find out how many boroughs and neighborhoods are there in Toronto:

In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


## Use geopy library to get the latitude and longitude values of Toronto

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.

In [ ]:
!pip install geocoder

In [29]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [37]:
import geocoder

In [38]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M1B')

[43.80862623100006, -79.18991284599997]

In [40]:
postal_codes = neighborhoods['PostalCode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [41]:
neighborhoods_coord = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
neighborhoods['Latitude'] = neighborhoods_coord['Latitude']
neighborhoods['Longitude'] = neighborhoods_coord['Longitude']
neighborhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


Create a map of Toronto with neighborhoods superimposed on top.

In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        #parse_html=False).
        #.add_to(map_toronto)  
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [43]:
neighborhoods.to_csv('coord_Toronto.csv')